In [1]:
import duckdb
from pathlib import Path
import pandas as pd
from sentence_transformers import SentenceTransformer
from typing import List, Any
from IPython.display import Markdown as md

/Users/serickson-local/src/ucsb-writing-data/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
# source data: survey responses from students.
df = pd.read_csv(Path("data") / 'reflections.csv') 

# load the model for computing embeddings for query string
model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")

# embeddings for the text responses in the survey data have been pre-computed
embeddings_url = "https://dreamlab-public.s3.us-west-2.amazonaws.com/sorapure/mxbai_embeddings.parquet"
embeddings_file = Path("outputs") / 'mxbai_embeddings.parquet'

# download embeddings file if the file doesn't exist
if not embeddings_file.exists():
    duckdb.execute(f"COPY (SELECT * from read_parquet('{embeddings_url}')) TO '{embeddings_file}' (FORMAT PARQUET);")

In [15]:
def search_df(q: str, limit: int = 10) -> List[Any]:
    query = f"Represent this sentence for searching relevant passages: {q}"
    query_embed = model.encode(query)
    sql = f"""
        FROM read_parquet('{embeddings_file}')
        SELECT 
            student_id as perm,
            question_id,
            array_distance(
                CAST(embedding as FLOAT[1024]),
                CAST($embed as FLOAT[1024])
            ) AS distance
        ORDER BY distance ASC
        LIMIT {limit};
    """
    result = duckdb.execute(sql, {"embed": query_embed}).fetch_df()
    result['text'] = pd.merge(left=result, right=df, on="perm").apply(lambda row: row[row['question_id']], axis=1)
    return result

def search_display(q: str, limit: int = 10):
    result = search_df(q, limit)
    for r in result['text']:
        if type(r) is str:
            display(md(r))


In [18]:
search_display("athletics and sports")

I would relate most to the "Gender Stereotypes and Learning Approaches in Computer Science" by Chanchan Mao. In high school, I was a cheerleader. Whenever we would have tryouts or open gym I would be shocked to see that no boys were trying out. I talked to a couple of my male friends and asked if they knew why no males were trying out. They responded that it was a "female-only activity". This angered me I would consider periods or giving birth to be considered female-only activities, not sports. Yes, there are sports dedicated to a specific gender, such as women's volleyball or boys' basketball. But there are all sports that aren't specific at all such as football or cheerleading. Eventually, one day a boy Thomas decided to show up for the open gym and talk to him as to why he finally began to show up. He said he was intimated by a sport that was dominated by main women and would feel outcasted from the rest of the team. As a female who has stuck to sports dedicated to my gender, I could feel what Thomas was

I've had many writings assignments where I had to explain whether I agree or disagre with a set of beleifs and practices. During my gender studies course I was given the task to listen to any gender-related podcast and write an opinionated response. Being interested in sports and intruiged by transgender rights, I listened to podcasts about the controversy around transgender athletes. Should female athletes who were born as male be allowed to participate in female competitions? As of today, many states are stripping transgender athletes of the right to particiapte in any sports competition due to an unfair advantage. The simple act of playing a sport is one of the only activities nowadays that brings people together - no matter your beleifs. Sports have always been a prominent aspect of my life and at a certain age I truly believed it would be my profession. With my father being a D1 baseball player I was determined to follow in his footsteps and make him proud. At some point I knew the MLB was out of my reac

I thought both readings were well crafted and informative. I thoroughly enjoyed the second reading because I believe it is extremely important. I have written many papers, taken classes, and experienced things in life that have prepared me to understand such readings. At school here, I took the class comm88 research methods. The class was all about coming to end conclusions on various research questions that I would construct. Just like the second reading stressed, I would constantly revise my questions in order to broaden my talking points and explore deeper. Outside of the classroom, certain encounters and life experiences have also prepared me to question things on a day to day basis. Being a student-athlete for a couple of years and then stepping away from my sport allowed me to question first hand the effects of the consistent structure of a collegiate sport and its relationship with academic success. Experiences like this have prepped me for adequately read and understand put to use the topics of both r

These student writing samples are very similar to ones I've done in the past as at least one per semester, my high school required an argumentative essay be assigned to each student. Last year, I was able to choose a topic from over twenty controversial ideas that ranged from should uniforms be an enforced school policy to the effectiveness of honors/AP classes. I chose to explore the ideas of professional sports and the difference between the compensation men and women receive. I spent about a week researching professional athletes and sports reporters as well as reviewing the statistics over the years of payment received by players. I chose to focus on the issue that women are paid far less than men solely based on tradition and sexism. I pointed out that stereotypes assume men are far more successful than women, but these ideas are wrong. For example, Serena Williams has won more Grand Slams than any other tennis player ever, yet male tennis players gain over two times the amount with even less success. I

These assignments are vastly similar to previous assignments I have had in high school. What first comes to mind when I read about these assignments is the research paper I wrote in high school. I spent the whole year researching and writing a paper about the correlation between succeeding in academics and participating in long distance running. I had a fun time writing this paper because it was very interesting to me, as are these assignments. 

These student writing samples are not very similar to any writing that I have done in the past. Although I have responded to scholarly topics I believe that I would have a bit of a hard time applying a rhetorical approach to the reading and responding to it with my own personal research. Previous experiences in the classroom such as extensive research projects, and argumentative essays have given me a familiar presence around writing samples, such as the ones provided. Conducting research on why women sports are less popularized has helped familiarize myself with voicing my opinion to reflect on the misconceptions of why toxic misogynic affects the viewing of these women sports. This task has helped me understand the idea of pursuing an interest in a topic, and understand why their are many taboos regarding a sport. 

The subject "stereotypes" was mostly tackled during my rhetorical class. Each student had to make an argumentative speech about a topic that they liked and we had to "pretend" that we were a specific type of person. A big part of the assignment was to research what kinds of stereotypes we could face as the specific type of person. I choose to be me, a student athlete and I had in my speech I then had to break the barrier and shoe that the negative stereotypes about student athletes were false.

When I was in high school I was part of my school's soccer team for two years.

In India, girls aren't usually found playing soccer, or even if they are, the coaches would make sure that the guys went easy on them to make sure that the girls weren't hurt. I found this pretty insulting because I was in fact pretty tough and was quite good at the sport, better than most of the guys on the soccer team. 

I made sure that our team wouldn't go unnoticed and our coach was quite impressed with what we were capable of. We eventually won several local tournaments and got qualified for nationals where we came in second.

During one of my unit tests in English, I was given a topic where I had to write an essay about gender stereotypes and whether India was deteriorating or improving based off that fact. I wrote about my experience being in the soccer team as a girl.

So I would say, I'm fairly familiar with the student writing samples mentioned above.

The writing sample by ChanChan Mao is very similar to some of the writing that I've done in the past. For example, in one of my English class, we discussed gender norms and how as a changing society we are slowly breaking out of those norms. We researched the gender wage gap, job opportunity disparities as well as stereotypes placed on both men and women in our modern society. For my paper, I wrote about how volleyball is usually seen as a female-dominated sport here in America, with very few men playing the sport in comparison. I researched online for statistics gathered on the international popularity of the sport between men and women, the objective skill difference between the two in pro-league tournaments, and much more. I cited my evidence at the very end of the paper and compared my findings with my classmates' findings on their respective topics. The type of paper that "It's All in the Pedagogy" identifies as is one that I'm less familiar with but no less interested in learning how to imitate. While t

Firstly, the "Memory Project" would be the easiest assignment to me because in ninth grade English honours I had an assignment very similar to this one. In this assignment I wrote about my 18 and under boys water polo practice that I went to as a 13 year old girl who weighed no more than 110 pounds. I related this situation to my identity today because it taught me to not be intimidated by individuals who are more powerful than I am. Although I did not have to site a secondary source, and epigraph or provide a picture, I did have to site a primary source. My primary source was the coach of the 18 and under boys who initially wanted me to practice with his team. 

During my time in AP Lang. and Comp. and AP Literature, I have written an extensive amount of response essays that required an audience, prewriting, writing strategies and a thesis that connects to the whole essay. For example, during AP Lang. and Comp. one of our assignments were to write a response essay on "The Plastic Pink Flamingo". 

Rhetorical